In [1]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')

[nltk_data] Downloading package punkt to /home/abel/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /home/abel/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /home/abel/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/abel/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/abel/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.


True

In [2]:
import pandas as pd
import numpy as np

import pickle as pk

In [5]:

import warnings
warnings.filterwarnings("ignore")


from bs4 import BeautifulSoup
import unicodedata
import re

from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

from nltk.corpus import stopwords


from nltk.corpus import wordnet
from nltk import pos_tag
from nltk import ne_chunk

from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from nltk.probability import FreqDist
import matplotlib.pyplot as plt
from wordcloud import WordCloud

In [7]:
df = pd.read_csv('Amazon_Unlocked_Mobile_small.csv')
df.head()

,Unnamed: 0,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
0,0,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,I feel so LUCKY to have found this used (phone...,1.0
1,1,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,"nice phone, nice up grade from my pantach revu...",0.0
2,2,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,Very pleased,0.0
3,3,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,It works good but it goes slow sometimes but i...,0.0
4,4,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,Great phone to replace my lost phone. The only...,0.0


In [8]:
df = df[['Rating', 'Reviews']]
df.head()

,Rating,Reviews
0,5,I feel so LUCKY to have found this used (phone...
1,4,"nice phone, nice up grade from my pantach revu..."
2,5,Very pleased
3,4,It works good but it goes slow sometimes but i...
4,4,Great phone to replace my lost phone. The only...


In [9]:
df['Reviews'].iloc[0]

"I feel so LUCKY to have found this used (phone to us & not used hard at all), phone on line from someone who upgraded and sold this one. My Son liked his old one that finally fell apart after 2.5+ years and didn't want an upgrade!! Thank you Seller, we really appreciate it & your honesty re: said used phone.I recommend this seller very highly & would but from them again!!"

In [10]:
df.dtypes

Rating      int64
Reviews    object
dtype: object

In [12]:
#To be on the safe side, I convert the reviews as strings to be able to work with them correctly.
df['Reviews'] = df['Reviews'].astype(str)
df['Reviews']

0        I feel so LUCKY to have found this used (phone...
1        nice phone, nice up grade from my pantach revu...
2                                             Very pleased
3        It works good but it goes slow sometimes but i...
4        Great phone to replace my lost phone. The only...
                               ...                        
19995                            had some technical issues
19996    Works very well. Shipped very quickly and is s...
19997    Phone looked brand new. I've had it one month ...
19998                Doesn't hold a charge and dies at 25%
19999    I've had this phone for over a month now and I...
Name: Reviews, Length: 20000, dtype: object

# 3 Definition of required Functions
All functions are summarized here. I will show them again in the course of this post at the place where they are used.



In [13]:
def remove_html_tags_func(text):
    '''
    Removes HTML-Tags from a string, if present
    
    Args:
        text (str): String to which the function is to be applied, string
    
    Returns:
        Clean string without HTML-Tags
    ''' 
    return BeautifulSoup(text, 'html.parser').get_text()

In [14]:
def remove_url_func(text):
    '''
    Removes URL addresses from a string, if present
    
    Args:
        text (str): String to which the function is to be applied, string
    
    Returns:
        Clean string without URL addresses
    ''' 
    return re.sub(r'https?://\S+|www\.\S+', '', text)

In [15]:
def remove_accented_chars_func(text):
    '''
    Removes all accented characters from a string, if present
    
    Args:
        text (str): String to which the function is to be applied, string
    
    Returns:
        Clean string without accented characters
    '''
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')

In [16]:
def remove_punctuation_func(text):
    '''
    Removes all punctuation from a string, if present
    
    Args:
        text (str): String to which the function is to be applied, string
    
    Returns:
        Clean string without punctuations
    '''
    return re.sub(r'[^a-zA-Z0-9]', ' ', text)

In [17]:
def remove_irr_char_func(text):
    '''
    Removes all irrelevant characters (numbers and punctuation) from a string, if present
    
    Args:
        text (str): String to which the function is to be applied, string
    
    Returns:
        Clean string without irrelevant characters
    '''
    return re.sub(r'[^a-zA-Z]', ' ', text)

In [18]:
def remove_extra_whitespaces_func(text):
    '''
    Removes extra whitespaces from a string, if present
    
    Args:
        text (str): String to which the function is to be applied, string
    
    Returns:
        Clean string without extra whitespaces
    ''' 
    return re.sub(r'^\s*|\s\s*', ' ', text).strip()

In [19]:
def word_count_func(text):
    '''
    Counts words within a string
    
    Args:
        text (str): String to which the function is to be applied, string
    
    Returns:
        Number of words within a string, integer
    ''' 
    return len(text.split())

# 4 Text Pre-Processing
There are some text pre-processing steps to consider and a few more you can do. In this post I will talk about text cleaning.

## 4.1 Text Cleaning
Here I have created an example string, where you can understand the following steps very well.

In [20]:
messy_text = \
"Hi e-v-e-r-y-o-n-e !!!@@@!!! I gave a 5-star rating. \
Bought this special product here: https://www.amazon.com/. Another link: www.amazon.com/ \
Here the HTML-Tag as well:  <a href='https://www.amazon.com/'> …</a>. \
I HIGHLY RECOMMEND THIS PRDUCT !! \
I @ (love) [it] <for> {all} ~it's* |/ #special / ^^characters;! \
I am currently investigating the special device and am excited about the features. Love it! \
Furthermore, I found the support really great. Paid about 180$ for it (5.7inch version, 4.8'' screen). \
Sómě special Áccěntěd těxt and words like résumé, café or exposé.\
"
messy_text

"Hi e-v-e-r-y-o-n-e !!!@@@!!! I gave a 5-star rating. Bought this special product here: https://www.amazon.com/. Another link: www.amazon.com/ Here the HTML-Tag as well:  <a href='https://www.amazon.com/'> …</a>. I HIGHLY RECOMMEND THIS PRDUCT !! I @ (love) [it] <for> {all} ~it's* |/ #special / ^^characters;! I am currently investigating the special device and am excited about the features. Love it! Furthermore, I found the support really great. Paid about 180$ for it (5.7inch version, 4.8'' screen). Sómě special Áccěntěd těxt and words like résumé, café or exposé."

In the following I will perform the individual steps for text cleaning and always use parts of the messy_text string.

4.1.1 Conversion to Lower Case
In general, it is advisable to format the text completely in lower case.

In [21]:
messy_text_lower_case = \
"I HIGHLY RECOMMEND THIS PRDUCT !!\
"
messy_text_lower_case

'I HIGHLY RECOMMEND THIS PRDUCT !!'

In [22]:
messy_text_lower_case.lower()

'i highly recommend this prduct !!'

In [23]:
messy_text_html = \
"Here the HTML-Tag as well:  <a href='https://www.amazon.com/'> …</a>.\
"
messy_text_html

"Here the HTML-Tag as well:  <a href='https://www.amazon.com/'> …</a>."

In [24]:
remove_html_tags_func(messy_text_html)

'Here the HTML-Tag as well:   ….'

In [25]:
df['Clean_Reviews'] = df['Reviews'].str.lower()
df['Clean_Reviews'] = df['Clean_Reviews'].apply(remove_html_tags_func)
df['Clean_Reviews'] = df['Clean_Reviews'].apply(remove_url_func)
df['Clean_Reviews'] = df['Clean_Reviews'].apply(remove_accented_chars_func)
df['Clean_Reviews'] = df['Clean_Reviews'].apply(remove_punctuation_func)
df['Clean_Reviews'] = df['Clean_Reviews'].apply(remove_irr_char_func)
df['Clean_Reviews'] = df['Clean_Reviews'].apply(remove_extra_whitespaces_func)

df.head()

,Rating,Reviews,Clean_Reviews
0,5,I feel so LUCKY to have found this used (phone...,i feel so lucky to have found this used phone ...
1,4,"nice phone, nice up grade from my pantach revu...",nice phone nice up grade from my pantach revue...
2,5,Very pleased,very pleased
3,4,It works good but it goes slow sometimes but i...,it works good but it goes slow sometimes but i...
4,4,Great phone to replace my lost phone. The only...,great phone to replace my lost phone the only ...


In [26]:
df['Word_Count'] = df['Clean_Reviews'].apply(word_count_func)

df[['Clean_Reviews', 'Word_Count']].head()

,Clean_Reviews,Word_Count
0,i feel so lucky to have found this used phone ...,70
1,nice phone nice up grade from my pantach revue...,40
2,very pleased,2
3,it works good but it goes slow sometimes but i...,17
4,great phone to replace my lost phone the only ...,44


In [27]:
print('Average of words counted: ' + str(df['Word_Count'].mean()))

Average of words counted: 34.73695


In [28]:
# other example
messy_text = \
"Hi e-v-e-r-y-o-n-e !!!@@@!!! I gave a 5-star rating. \
Bought this special product here: https://www.amazon.com/. Another link: www.amazon.com/ \
Here the HTML-Tag as well:  <a href='https://www.amazon.com/'> …</a>. \
I HIGHLY RECOMMEND THIS PRDUCT !! \
I @ (love) [it] <for> {all} ~it's* |/ #special / ^^characters;! \
I am currently investigating the special device and am excited about the features. Love it! \
Furthermore, I found the support really great. Paid about 180$ for it (5.7inch version, 4.8'' screen). \
Sómě special Áccěntěd těxt and words like résumé, café or exposé.\
"
messy_text

"Hi e-v-e-r-y-o-n-e !!!@@@!!! I gave a 5-star rating. Bought this special product here: https://www.amazon.com/. Another link: www.amazon.com/ Here the HTML-Tag as well:  <a href='https://www.amazon.com/'> …</a>. I HIGHLY RECOMMEND THIS PRDUCT !! I @ (love) [it] <for> {all} ~it's* |/ #special / ^^characters;! I am currently investigating the special device and am excited about the features. Love it! Furthermore, I found the support really great. Paid about 180$ for it (5.7inch version, 4.8'' screen). Sómě special Áccěntěd těxt and words like résumé, café or exposé."

In [29]:
# quitar mayusculas
messy_text_lower = messy_text.lower()
messy_text_lower

"hi e-v-e-r-y-o-n-e !!!@@@!!! i gave a 5-star rating. bought this special product here: https://www.amazon.com/. another link: www.amazon.com/ here the html-tag as well:  <a href='https://www.amazon.com/'> …</a>. i highly recommend this prduct !! i @ (love) [it] <for> {all} ~it's* |/ #special / ^^characters;! i am currently investigating the special device and am excited about the features. love it! furthermore, i found the support really great. paid about 180$ for it (5.7inch version, 4.8'' screen). sómě special áccěntěd těxt and words like résumé, café or exposé."

In [30]:
# remover htlm
messy_text_wo_html = remove_html_tags_func(messy_text_lower)
messy_text_wo_html

"hi e-v-e-r-y-o-n-e !!!@@@!!! i gave a 5-star rating. bought this special product here: https://www.amazon.com/. another link: www.amazon.com/ here the html-tag as well:   …. i highly recommend this prduct !! i @ (love) [it]  {all} ~it's* |/ #special / ^^characters;! i am currently investigating the special device and am excited about the features. love it! furthermore, i found the support really great. paid about 180$ for it (5.7inch version, 4.8'' screen). sómě special áccěntěd těxt and words like résumé, café or exposé."

In [31]:
## remover url
messy_text_wo_url = remove_url_func(messy_text_wo_html)
messy_text_wo_url

"hi e-v-e-r-y-o-n-e !!!@@@!!! i gave a 5-star rating. bought this special product here:  another link:  here the html-tag as well:   …. i highly recommend this prduct !! i @ (love) [it]  {all} ~it's* |/ #special / ^^characters;! i am currently investigating the special device and am excited about the features. love it! furthermore, i found the support really great. paid about 180$ for it (5.7inch version, 4.8'' screen). sómě special áccěntěd těxt and words like résumé, café or exposé."

In [33]:
# remover acentos
messy_text_wo_acc_chars = remove_accented_chars_func(messy_text_wo_url)
messy_text_wo_acc_chars

"hi e-v-e-r-y-o-n-e !!!@@@!!! i gave a 5-star rating. bought this special product here:  another link:  here the html-tag as well:   .... i highly recommend this prduct !! i @ (love) [it]  {all} ~it's* |/ #special / ^^characters;! i am currently investigating the special device and am excited about the features. love it! furthermore, i found the support really great. paid about 180$ for it (5.7inch version, 4.8'' screen). some special accented text and words like resume, cafe or expose."

In [35]:
# remueve caracteres especiales
messy_text_wo_punct = remove_punctuation_func(messy_text_wo_acc_chars)
messy_text_wo_punct

'hi e v e r y o n e           i gave a 5 star rating  bought this special product here   another link   here the html tag as well         i highly recommend this prduct    i    love   it    all   it s      special     characters   i am currently investigating the special device and am excited about the features  love it  furthermore  i found the support really great  paid about 180  for it  5 7inch version  4 8   screen   some special accented text and words like resume  cafe or expose '

In [37]:
# remueve numeros
messy_text_wo_irr_char = remove_irr_char_func(messy_text_wo_punct)
messy_text_wo_irr_char

'hi e v e r y o n e           i gave a   star rating  bought this special product here   another link   here the html tag as well         i highly recommend this prduct    i    love   it    all   it s      special     characters   i am currently investigating the special device and am excited about the features  love it  furthermore  i found the support really great  paid about      for it     inch version        screen   some special accented text and words like resume  cafe or expose '

In [39]:
# remueve espacios extra
clean_text = remove_extra_whitespaces_func(messy_text_wo_irr_char)
clean_text

'hi e v e r y o n e i gave a star rating bought this special product here another link here the html tag as well i highly recommend this prduct i love it all it s special characters i am currently investigating the special device and am excited about the features love it furthermore i found the support really great paid about for it inch version screen some special accented text and words like resume cafe or expose'

In [40]:
print('Number of words: ' + str(word_count_func(clean_text)))

Number of words: 80
